#Personal Medical Comprehend

This projects focus will be around gaming specific insights into the medical terminology and discussions around what is happening to a specific patient from unstructured medical text - 

Specifically the projects goals are to build out a tool that can pull out all the medical related information from certain data and go a step further and do forms of basic outcome assessments and NER on that data using some advanced tooling from Sci Spacey Bio BERT and Stanza. 

Finally break down all findings into structured data and include sentiment analysis for a general patient outcome prediction either positive or negative.


This tool could be used on individual text targets to extract the afore mentioned insights and in the future could be used on clusters of targets to begin to map specific outcomes for a patient across time.



## Step One - Extracting Medical Text - Sci Spacy 
First lets use Sci Spacy to extract all relivent medical text from our central target text example - 

### Requirements
Python 3 pandas spacy>=3.0 scispacy You can simply pip install all of them.

We also need to download and install the NER model from scispaCy. To install the en_ner_bc5cdr_md model use the following command:

In [ ]:
!pip install scispacy

     |████████████████████████████████| 44 kB 1.4 MB/s 
     |████████████████████████████████| 13.5 MB 17.1 MB/s 
     |████████████████████████████████| 71 kB 5.8 MB/s 
     |████████████████████████████████| 6.0 MB 34.9 MB/s 
     |████████████████████████████████| 188 kB 45.6 MB/s 
     |████████████████████████████████| 42 kB 911 kB/s 
     |████████████████████████████████| 457 kB 50.6 MB/s 
     |████████████████████████████████| 181 kB 55.1 MB/s 
     |████████████████████████████████| 10.1 MB 48.3 MB/s 
     |████████████████████████████████| 653 kB 39.5 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: s

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

     |████████████████████████████████| 125.1 MB 27 kB/s 
     |████████████████████████████████| 5.8 MB 14.2 MB/s 
  Created wheel for en-ner-bc5cdr-md: filename=en_ner_bc5cdr_md-0.4.0-py3-none-any.whl size=125666883 sha256=b16190e2dd4e4aacb13b63aca1b879da11c644b00740f88402677773901e462c
  Stored in directory: /root/.cache/pip/wheels/6a/17/04/ed8f50ffdcb9b9e4ce601c0c3cf88361f2b3f54a10c0fa0803
Successfully built en-ner-bc5cdr-md
  Attempting uninstall: typer
    Found existing installation: typer 0.4.1
    Uninstalling typer-0.4.1:
      Successfully uninstalled typer-0.4.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.4
    Uninstalling spacy-3.2.4:
      Successfully uninstalled spacy-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scispacy 0.5.0 requires spacy<3.3.0,>=3.2.0, but you have spacy 3.0.8 which is incompatible.


Now lets take some medical example text and work to Identify its key componets 

In [ ]:
Text ='The patient had a dry cough and fever, they were treated with Paracetamol and saw major improvements in the next few weeks'

Now that we have are text import in the tools for Bio based NER that we need -

In [ ]:
import scispacy
import spacy
nlp = spacy.load("en_ner_bc5cdr_md") 

spacy.load will return a Language object containing all components and data needed to process text. This object is usually called nlp in the documentation and tutorials. Calling the nlp object on a string of text will return a processed Doc object with the text split into words and annotated.

Let’s get all identified entities and print their text, start position, end position, and type:

In [ ]:
doc = nlp(Text)
print("TEXT", "START", "END", "ENTITY TYPE")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

TEXT START END ENTITY TYPE
dry cough 18 27 DISEASE
fever 32 37 DISEASE
Paracetamol 62 73 CHEMICAL


Next We Will Impliment Stanza

Stanza is a Python NLP toolkit that supports 60+ human languages. It is built with highly accurate neural network components that enable efficient training and evaluation with your own annotated data, and offers pretrained models on 100 treebanks. Additionally, Stanza provides a stable, officially maintained Python interface to Java Stanford CoreNLP Toolkit.

Note that Stanza only supports Python 3.6 and above. Installing and importing Stanza are as simple as running the following commands:

In [ ]:
!pip install Stanza

     |████████████████████████████████| 432 kB 15.0 MB/s 
     |████████████████████████████████| 175 kB 36.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=b8c6657fc4078835c508867c170e385129b69b98b672eac8f06394150dc30e9d
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


Next download the english model 

In [ ]:
# Import the package
import stanza

# Download an English model into the default directory
print("Downloading English model...")
stanza.download('en')

2022-04-22 08:36:39 INFO: Downloading default packages for language: en (English)...


2022-04-22 08:36:52 INFO: Finished downloading models and saved to /root/stanza_resources.


#3. Processing Text
Constructing Pipeline To process a piece of text, you'll need to first construct a Pipeline with different Processor units. The pipeline is language-specific, so again you'll need to first specify the language (see examples).

In [ ]:
# Build an English pipeline, with all processors by default
print("Building an English pipeline...")
en_nlp = stanza.Pipeline('en',verbose=True)

2022-04-22 08:36:52 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-04-22 08:36:52 INFO: Use device: cpu
2022-04-22 08:36:52 INFO: Loading: tokenize
2022-04-22 08:36:52 INFO: Loading: pos


Building an English pipeline...


2022-04-22 08:36:52 INFO: Loading: lemma
2022-04-22 08:36:52 INFO: Loading: depparse
2022-04-22 08:36:53 INFO: Loading: sentiment
2022-04-22 08:36:53 INFO: Loading: constituency
2022-04-22 08:36:54 INFO: Loading: ner
2022-04-22 08:36:56 INFO: Done loading processors!


#Annotating Text
After a pipeline is successfully constructed, you can get annotations of a piece of text simply by passing the string into the pipeline object. The pipeline will return a Document object, which can be used to access detailed annotations from. For example:

In [ ]:
# Processing English text
Text ='The patient had a dry cough and fever, they were treated with Paracetamol and saw major improvements in the next few weeks'

en_doc = en_nlp(Text)
print(type(en_doc))

<class 'stanza.models.common.doc.Document'>


We can now have the model perform a full prediction on the structure and anotation of the text- 

In [ ]:
for i, sent in enumerate(en_doc.sentences):
    print("[Sentence {}]".format(i+1))

    for word in sent.words:
        print("{:12s}\t{:12s}\t{:6s}\t{:d}\t{:12s}".format(\
              word.text, word.lemma, word.pos, word.head, word.deprel))
        
    print("")

[Sentence 1]
The         	the         	DET   	2	det         
patient     	patient     	NOUN  	3	nsubj       
had         	have        	VERB  	0	root        
a           	a           	DET   	6	det         
dry         	dry         	ADJ   	6	amod        
cough       	cough       	NOUN  	3	obj         
and         	and         	CCONJ 	8	cc          
fever       	fever       	NOUN  	6	conj        
,           	,           	PUNCT 	3	punct       
they        	they        	PRON  	12	nsubj:pass  
were        	be          	AUX   	12	aux:pass    
treated     	treat       	VERB  	3	parataxis   
with        	with        	ADP   	14	case        
Paracetamol 	Paracetamol 	PROPN 	12	obl         
and         	and         	CCONJ 	16	cc          
saw         	see         	VERB  	12	conj        
major       	major       	ADJ   	18	amod        
improvements	improvement 	NOUN  	16	obj         
in          	in          	ADP   	23	case        
the         	the         	DET   	23	det         
next        	next

#Now lets perform Basic NER 
Running the NERProcessor simply requires the TokenizeProcessor. After the pipeline is run, the Document will contain a list of Sentences, and the Sentences will contain lists of Tokens. Named entities can be accessed through Document or Sentence’s properties entities or ents. Alternatively, token-level NER tags can be accessed via the ner fields of Token.

Accessing Named Entities for Sentence and Document Here is an example of performing named entity recognition for a piece of text and accessing the named entities in the entire document:

In [ ]:
Text ='The patient had a dry cough and fever, they were treated with Paracetamol and saw major improvements in the next few weeks'


nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')

doc = nlp(Text)
print()
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')


2022-04-22 08:36:58 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2022-04-22 08:36:58 INFO: Use device: cpu
2022-04-22 08:36:58 INFO: Loading: tokenize
2022-04-22 08:36:58 INFO: Loading: ner
2022-04-22 08:36:59 INFO: Done loading processors!



entity: Paracetamol	type: PRODUCT
entity: the next few weeks	type: DATE


## Now lets use Stanzas medical libs to define our text further

In [ ]:
stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
nlp = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'})

doc = nlp(Text)

# print out the entities
for ent in doc.entities:
    print(f'{ent.text}\t{ent.type}') 

2022-04-22 09:10:12 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package |
-----------------------------
| tokenize        | mimic   |
| pos             | mimic   |
| lemma           | mimic   |
| depparse        | mimic   |
| ner             | i2b2    |
| backward_charlm | mimic   |
| forward_charlm  | mimic   |
| pretrain        | mimic   |



2022-04-22 09:10:21 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-04-22 09:10:21 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |
| lemma     | mimic   |
| depparse  | mimic   |
| ner       | i2b2    |

2022-04-22 09:10:21 INFO: Use device: cpu
2022-04-22 09:10:21 INFO: Loading: tokenize
2022-04-22 09:10:21 INFO: Loading: pos
2022-04-22 09:10:22 INFO: Loading: lemma
2022-04-22 09:10:22 INFO: Loading: depparse
2022-04-22 09:10:22 INFO: Loading: ner
2022-04-22 09:10:23 INFO: Done loading processors!


a dry cough	PROBLEM
fever	PROBLEM
Paracetamol	TREATMENT


## Final step outcome prediction 
Now that we have our text fully processed we can take the final step in our applications analytics features and generate a sentiment based prediction on outcome - 

This is going to be basic at first but can be advanced in the future to start with we are just going to access the doctors general sentiment in the note and use that as an indicator of a postive outcome or a negtive one.

</br>

To do this we will use the Roberta sentiment analysis model - 

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 20.8 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 47.1 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 6.6 MB 25.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# import what is needed from libs
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [ ]:
#load in the target model 
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

print('model loaded')

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

model loaded


In [ ]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [ ]:
# download and save the model 
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# -tune or retrain the model here and then save with command bellow 

#save the model - put in name of folder in instead to save to target location
model.save_pretrained(MODEL)

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [ ]:
# example of model use 
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
text ='The patient had a dry cough and fever, they were treated with Paracetamol and saw major improvements in the next few weeks'
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores

ranking = np.argsort(scores)
ranking = ranking[::-1]
print('Docters Note:')
print(Text)
print()

print('Patient Outcome Assessment:')
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

Docters Note:
The patient had a dry cough and fever, they were treated with Paracetamol and saw major improvements in the next few weeks

Patient Outcome Assessment:
1) positive 0.7764
2) neutral 0.2131
3) negative 0.0105


#Dataset
for further work use Unstructured medical data, like medical transcriptions. Now these are pretty hard to find but we have a good set. Here we are using a medical transcription dataset scraped from the MTSamples website by Tara Boyle and made available at Kaggle.

https://www.kaggle.com/tboyle10/medicaltranscriptions